# decorators

In [1]:
def do_nothing(f):
    print(f)
    return f

In [2]:
# decorator syntax

@do_nothing
def greet():
    return "hello"

<function greet at 0x10a8280d0>


In [3]:
# same as ("syntactic" sugar for)

def greet():
    return "hello"

greet = do_nothing(greet)

<function greet at 0x10a828dc0>


In [4]:
# decorator can return anything

def two(f):
    return 2

@two
def add(x, y):
    return x + y

add

2

## example: timer

In [5]:
# most often, returns a modified function

def timer(func):
    from datetime import datetime
    def new_function(*args):
        start = datetime.now()
        result = func(*args)
        print('function call took', datetime.now() - start)
        return result
    
    return new_function

@timer
def add_up(n):
    return sum(range(1, n + 1))

In [6]:
add_up

<function __main__.timer.<locals>.new_function(*args)>

In [7]:
add_up(10**7)

function call took 0:00:00.355754


50000005000000

In [8]:
# variadic arguments:

def f(*nums):
    print(nums)
    return sum(nums)

f(1, 2, 3)
# same as
f(*[1, 2, 3])

(1, 2, 3)
(1, 2, 3)


6

In [9]:
# builtin example
max(*[1, 2, 0])

2

## example: memoization

In [10]:
def fib(n):
    if n < 2:
        return 1
    return fib(n - 2) + fib(n - 1)

In [11]:
for n in range(10):
    print(fib(n), end=', ')

1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 

In [12]:
%%time
fib(35)

CPU times: user 4.51 s, sys: 12.6 ms, total: 4.52 s
Wall time: 4.55 s


14930352

In [13]:
# problem: exponential number of mostly redundant calls
# solution: cache results

cache = {}
def fib(n):
    if n in cache:
        return cache[n]
    
    if n < 2:
        return 1
    
    cache[n] = fib(n - 2) + fib(n - 1)
    return cache[n]

In [14]:
%%time
fib(35)

CPU times: user 49 µs, sys: 1 µs, total: 50 µs
Wall time: 55.1 µs


14930352

In [15]:
# problem: caching and fibonacci are separate concerns, and what if you want to cache other functions
# solution: factor out caching into a decorator

def memoize(func):
    cache = {}
    
    def inner(*args):
        if args in cache:
            return cache[args]
        cache[args] = func(*args)
        return cache[args]

    return inner

@memoize
def fib(n):
    if n < 2:
        return 1
    return fib(n - 2) + fib(n - 1)

In [16]:
%time
fib(35)

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 10 µs


14930352

In [17]:
# where did the original function go?

fib.__closure__

(<cell at 0x10a870310: dict object at 0x10a8761c0>,
 <cell at 0x10a870490: function object at 0x10a86c790>)

## builtin decorator: `property`

In [18]:
class Person:
    def __init__(self, first, last):
        self.first = first
        self.last = last
        
    def full(self):
        return f'{self.first} {self.last}'

In [19]:
emmy = Person('Emmy', 'Noether')
print(emmy.first)
print(emmy.last)
print(emmy.full())

Emmy
Noether
Emmy Noether


In [20]:
# problem: full should just be an attribute, but based on first and last
# solution: the `property` decorator

class Person:
    def __init__(self, first, last):
        self.first = first
        self.last = last
    
    @property
    def full(self):
        return f'{self.first} {self.last}'

In [21]:
emmy = Person('Emmy', 'Noether')
print(emmy.first)
print(emmy.last)
print(emmy.full)

Emmy
Noether
Emmy Noether


In [22]:
# can also add setter

class Person:
    def __init__(self, first, last):
        self.first = first
        self.last = last
    
    @property
    def full(self):
        return f'{self.first} {self.last}'
    
    @full.setter
    def full(self, value):
        self.first, self.last = value.strip().split()

In [23]:
eric = Person('Eric', 'Blair')
eric.full = 'George Orwell'
print(eric.first)
print(eric.last)

George
Orwell


``property`` is an example of a "descriptor". You can make custom descriptors, and they enable frameworks such as SQLAlchemy.

# lambda

In [24]:
# a bad but succinct implementation of quicksort:

qsort = lambda lst: [] if not lst else \
        qsort([n for n in lst[:-1] if n < lst[-1]]) \
        + [lst[-1]] \
        + qsort([n for n in lst[:-1] if n >= lst[-1]])

# Python's "ternary" expression:
# [expression] if [condition] else [expression]

In [25]:
from random import randint

In [26]:
lst = [randint(-10,10) for _ in range(10)]
lst

[0, -3, 2, -1, 6, 8, 9, -5, 5, 4]

In [27]:
qsort(lst)

[-5, -3, -1, 0, 2, 4, 5, 6, 8, 9]

"qsort" is a name, so ``qsort`` is not anonymous. Anonymous recursive functions are possible. For example, see

https://scotchka.github.io/blog/html/2020/06/28/a_recursive_function_that_does_not_call_itself.html

# generators

In [28]:
def count():
    print('hello')
    yield 0
    print('world')
    yield 1
    print('!')
    yield 2

In [29]:
c = count()
c

<generator object count at 0x10a86af20>

In [30]:
next(c)

hello


0

In [31]:
def count(n=None):
    i = 0
    while not n or i < n:
        yield i
        i += 1
        
for num in count(5):
    print(num)

0
1
2
3
4


In [32]:
# no upper bound
c = count()
next(c)

0

## example: tree traversal

In [33]:
class Node:
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right
        
    @classmethod
    def make_tree(cls, lst):
        """Make a balanced binary tree."""
        if not lst:
            return None
        lst = sorted(lst)
        
        mid = len(lst) // 2
        
        node = cls(lst[mid])
        
        node.left = cls.make_tree(lst[:mid])
        node.right = cls.make_tree(lst[mid+1:])
        
        return node
    
    def __iter__(self):
        if self.left:
            for node in self.left:
                yield node
                
        yield self.value
        
        if self.right:
            for node in self.right:
                yield node
    

In [34]:
lst = list(range(25))

bst = Node.make_tree(lst)

for value in bst:
    print(value, end=', ')

0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 